In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.appName('LeetCode').getOrCreate()

24/12/23 15:20:31 WARN Utils: Your hostname, madiv resolves to a loopback address: 127.0.1.1; using 192.168.1.11 instead (on interface wlo1)
24/12/23 15:20:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/23 15:20:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sparkDataset = spark.read.option('header', True).option('delimiter', ',').csv('Dataset.csv')
pandasDataset = pd.read_csv('Dataset.csv')

sparkDataset.show()
pandasDataset.head()

+-----------+---------+-------+----------+------------+
|       name|continent|   area|population|         gdp|
+-----------+---------+-------+----------+------------+
|Afghanistan|     Asia| 652230|  25500100| 20343000000|
|    Albania|   Europe|  28748|   2831741| 12960000000|
|    Algeria|   Africa|2381741|  37100000|188681000000|
|    Andorra|   Europe|    468|     78115|  3712000000|
|     Angola|   Africa|1246700|  20609294|100990000000|
+-----------+---------+-------+----------+------------+



,name,continent,area,population,gdp
0,Afghanistan,Asia,652230,25500100,20343000000
1,Albania,Europe,28748,2831741,12960000000
2,Algeria,Africa,2381741,37100000,188681000000
3,Andorra,Europe,468,78115,3712000000
4,Angola,Africa,1246700,20609294,100990000000


### SQL

In [3]:
def big_countries(world: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    world.createOrReplaceTempView('World')
    sqlQuery = \
    '''
    SELECT
        name, population, area
    FROM World
    WHERE 
    population >= 25000000 OR
    area >= 3000000
    '''
    output = spark.sql(sqlQuery = sqlQuery)
    return output

output = big_countries(sparkDataset)
output.show()

+-----------+----------+-------+
|       name|population|   area|
+-----------+----------+-------+
|Afghanistan|  25500100| 652230|
|    Algeria|  37100000|2381741|
+-----------+----------+-------+



### Pyspark

In [4]:
def big_countries(world: pyspark.sql.dataframe.DataFrame) -> pyspark.sql.dataframe.DataFrame:
    output = world.filter((world.area >= 3000000)|
                          (world.population >= 25000000))\
                    .select(['name', 'population', 'area'])
    return output

output = big_countries(sparkDataset)
output.show()

+-----------+----------+-------+
|       name|population|   area|
+-----------+----------+-------+
|Afghanistan|  25500100| 652230|
|    Algeria|  37100000|2381741|
+-----------+----------+-------+



### Pandas

In [5]:
def big_countries(world: pd.DataFrame) -> pd.DataFrame:
    output = world[(world.area >= 3000000)|
                   (world.population >= 25000000)]
    output = output[['name', 'population', 'area']]
    output = pd.DataFrame(output)
    return output

output = big_countries(pandasDataset)
output.head()

,name,population,area
0,Afghanistan,25500100,652230
2,Algeria,37100000,2381741
